# Lecture 4 - Command Line Arguments

## 📕 Today's Agenda
---
 * [Simple argument parsing](#Simple-argument-parsing)
 * [Parsing with argparse](#Parsing-with-argparse)
 * [Script version and description](#Script-version-and-description)
 * [Mandatory and optional arguments](#Mandatory-and-optional-arguments)
 * [Actions and default values](#Actions-and-default-values)
 * [Argument validation](#Argument-validation)

## 🧪 Theory
---

### Simple argument parsing

One of the best feature of CLI tools is the ability of injecting data into a program. This
is a key point in automation world.
Parsing arguments can be messy when the number of arguments is big, but for one or two positional arguments
is enough to use `sys` module.

In [4]:
import sys

print(sys.argv)
print('First arg:', sys.argv[1])

for arg in sys.argv:
    print(arg)

['c:\\users\\mdinu\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\mdinu\\AppData\\Roaming\\jupyter\\runtime\\kernel-d3387e6a-dc92-4542-b796-ae65fa418cae.json']
First arg: -f
c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\ipykernel_launcher.py
-f
C:\Users\mdinu\AppData\Roaming\jupyter\runtime\kernel-d3387e6a-dc92-4542-b796-ae65fa418cae.json


### Parsing with argparse
The simplest and fastest path to a good argument parsing is usage of `argparse` module. It
provides a simple interface for argument handling if configured well.
It comes with out of the box support for common arguments like -h (--help) and -v (--version).

Let's jump in and create a simple arg parser.

NOTE: `parse_args` should be called without arguments when getting arguments from CLI. In this notebook
i can't pass arguments to the CLI so I will pass them into `parse_args` function call.


In [5]:
import argparse
argparser = argparse.ArgumentParser()
argparser.add_argument('name')
argparser.add_argument('surname')
args = argparser.parse_args(['mihai', 'dinu'])
print(args)
print(args.name)
print(args.surname)

Namespace(name='mihai', surname='dinu')
mihai
dinu


Code explained:
Line 1: create a new ArgumentParser object, this will hold info about available arguments.
Line 2: add first positional argument to the parser.
Line 3: add second positional argument to the parser.
Line 4: parse CLI args and store data into parser.
Line 5-7: Use data passed via arguments.

### Script version and description
As I already discussed about built-in ability to handle version and help, let's configure them.
Help is constructed using available arguments in parser. Use *help* kwarg to specify a help message for an argument.

In [6]:
argparser2 = argparse.ArgumentParser(description='Notebooks description.')
argparser2.add_argument('name', help='User name')
argparser2.parse_args(['--help'])

usage: ipykernel_launcher.py [-h] name

Notebooks description.

positional arguments:
  name        User name

optional arguments:
  -h, --help  show this help message and exit


SystemExit: 0

c:\users\mdinu\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


For version handling a there in not a already defined argument, we have to define it, instead exists an action.

In [ ]:
argparser2.add_argument('-v', action='version', version='1.0')
argparser2.parse_args(['-v'])

### Mandatory and optional arguments
Each optional argument can trigger one action or can pass a value with it. For example --verbose can set
a variable to True but if you provide a number after it (--verbose 2), the number can be used to set verbosity level.
To specify how much sub-arguments an argument can take `nargs` kwarg must be provided. Along `nargs` argument, `meta` kwarg
is needed, but not mandatory, to describe what is expected for that CLI arg.

In [ ]:
argparser3 = argparse.ArgumentParser()
argparser3.add_argument(
    '-n',
    nargs=1,
    metavar='NUMBERS',
    help="Generate n numbers."
)
argparser3.add_argument(
    '-r',
    '--read',
    help='Read files.'
)
args = argparser3.parse_args(['-h'])

### Actions and default values
As you already saw in script version, an action argument is needed to specify special actions. Argparse module puts in you use
more them one action. Using actions you can modify the default action of storing info in a variable after parsing.
Available actions:

- *store_const* - This stores the value specified by the const keyword argument. The *store_const* action is most commonly used with optional arguments that specify some sort of flag.
    `parser.add_argument('--foo', action='store_const', const=42)`
- *store_true* and *store_false* - These are special cases of *store_const* used for storing the values True and False respectively. In addition, they create default values of False and True respectively.
    `parser.add_argument('--foo', action='store_true')`
- *append* - This stores a list, and appends each argument value to the list. This is useful to allow an option to be specified multiple times.
    `parser.add_argument('--foo', action='append')`
- *count* - This counts the number of times a keyword argument occurs. For example, this is useful for increasing verbosity levels:
  ```python
    parser = argparse.ArgumentParser()
    parser.add_argument('--verbose', '-v', action='count', default=0)
    parser.parse_args(['-vvv'])
  ```

All optional arguments can be omitted and to avoid messy situations the default value comes in help. Set the `default` argument and
in case of no other value provided via CLI the default value will be used.

```python
    parser = argparse.ArgumentParser()
    parser.add_argument('--length', default=10)
    parser.add_argument('--width', default=10.5)
    parser.parse_args()
```
### Argument validation

Most of the times you need to make sure that the user input is as you expect. Argparse module give the possibility to validate
arguments for type or format. To add this ability, for each argument added you have to specify the `type` option.
The argument to type can be any callable that accepts a single string. If the function raises ArgumentTypeError, TypeError, or ValueError,
the exception is caught and a nicely formatted error message is displayed. No other exception types are handled.

Simple types:
- int
- float
- ascii
- ord
- open = file stream
- pathlib.Path = a valid path

```python
parser = argparse.ArgumentParser()
parser.add_argument('count', type=int)
parser.add_argument('distance', type=float)
parser.add_argument('street', type=ascii)
parser.add_argument('code_point', type=ord)
parser.add_argument('source_file', type=open)
parser.add_argument('dest_file', type=argparse.FileType('w', encoding='latin-1'))
parser.add_argument('datapath', type=pathlib.Path)
```

Of course, user defined functions can be used if they respect the rules presented above.

In [52]:
def validate_version(version):
    if not isinstance(version, str):
        raise TypeError('Expected a string.')
    v_parts = version.strip().split(".")
    if len(v_parts) != 3:
        raise ValueError("Expected format is xxx.xxx.xxx .")
    if not all([x.isdecimal() for x in v_parts]):
        raise ValueError("Version must contain only numbers.")
    return version.strip()

parser = argparse.ArgumentParser()
parser.add_argument("-v", type=validate_version, nargs=1)
pargs = parser.parse_args(['-v 2.2.2'])
print(pargs.v)

['2.2.2']


## 👩‍💻 Practice
---
1. Write a script that takes two arguments. First argument is a number and the second one is a path to an output file.
The script will compute Fibonacci numbers up to first argument and will write them on the output file
specified by the second argument, one number per line. Argument parsing will be done using *sys* module. Make sure you check
for the number of arguments provided by user and make sure that they are valid. For Fibonacci calculus make a generator function.

2. Modify the code written for point 1 in order to use `argparse`.

## 🏠 Homework
---